### DonorsChoose

<p>
DonorsChoose.org receives hundreds of thousands of project proposals each year for classroom projects in need of funding. Right now, a large number of volunteers is needed to manually screen each submission before it's approved to be posted on the DonorsChoose.org website.
</p>
<p>
    Next year, DonorsChoose.org expects to receive close to 500,000 project proposals. As a result, there are three main problems they need to solve:
<ul>
<li>
    How to scale current manual processes and resources to screen 500,000 projects so that they can be posted as quickly and as efficiently as possible</li>
    <li>How to increase the consistency of project vetting across different volunteers to improve the experience for teachers</li>
    <li>How to focus volunteer time on the applications that need the most assistance</li>
    </ul>
</p>    
<p>
The goal of the competition is to predict whether or not a DonorsChoose.org project proposal submitted by a teacher will be approved, using the text of project descriptions as well as additional metadata about the project, teacher, and school. DonorsChoose.org can then use this information to identify projects most likely to need further review before approval.
</p>

### About the DonorsChoose Data Set

The `train.csv` data set provided by DonorsChoose contains the following features:

Feature | Description 
----------|---------------
**`project_id`** | A unique identifier for the proposed project. **Example:** `p036502`   
**`project_title`**    | Title of the project. **Examples:**<br><ul><li><code>Art Will Make You Happy!</code></li><li><code>First Grade Fun</code></li></ul> 
**`project_grade_category`** | Grade level of students for which the project is targeted. One of the following enumerated values: <br/><ul><li><code>Grades PreK-2</code></li><li><code>Grades 3-5</code></li><li><code>Grades 6-8</code></li><li><code>Grades 9-12</code></li></ul>  
 **`project_subject_categories`** | One or more (comma-separated) subject categories for the project from the following enumerated list of values:  <br/><ul><li><code>Applied Learning</code></li><li><code>Care &amp; Hunger</code></li><li><code>Health &amp; Sports</code></li><li><code>History &amp; Civics</code></li><li><code>Literacy &amp; Language</code></li><li><code>Math &amp; Science</code></li><li><code>Music &amp; The Arts</code></li><li><code>Special Needs</code></li><li><code>Warmth</code></li></ul><br/> **Examples:** <br/><ul><li><code>Music &amp; The Arts</code></li><li><code>Literacy &amp; Language, Math &amp; Science</code></li>  
  **`school_state`** | State where school is located ([Two-letter U.S. postal code](https://en.wikipedia.org/wiki/List_of_U.S._state_abbreviations#Postal_codes)). **Example:** `WY`
**`project_subject_subcategories`** | One or more (comma-separated) subject subcategories for the project. **Examples:** <br/><ul><li><code>Literacy</code></li><li><code>Literature &amp; Writing, Social Sciences</code></li></ul> 
**`project_resource_summary`** | An explanation of the resources needed for the project. **Example:** <br/><ul><li><code>My students need hands on literacy materials to manage sensory needs!</code</li></ul> 
**`project_essay_1`**    | First application essay<sup>*</sup>  
**`project_essay_2`**    | Second application essay<sup>*</sup> 
**`project_essay_3`**    | Third application essay<sup>*</sup> 
**`project_essay_4`**    | Fourth application essay<sup>*</sup> 
**`project_submitted_datetime`** | Datetime when project application was submitted. **Example:** `2016-04-28 12:43:56.245`   
**`teacher_id`** | A unique identifier for the teacher of the proposed project. **Example:** `bdf8baa8fedef6bfeec7ae4ff1c15c56`  
**`teacher_prefix`** | Teacher's title. One of the following enumerated values: <br/><ul><li><code>nan</code></li><li><code>Dr.</code></li><li><code>Mr.</code></li><li><code>Mrs.</code></li><li><code>Ms.</code></li><li><code>Teacher.</code></li></ul>  
**`teacher_number_of_previously_posted_projects`** | Number of project applications previously submitted by the same teacher. **Example:** `2` 

See the section <b>Notes on the Essay Data</b> for more details about these features.

Additionally, the `resources.csv` data set provides more data about the resources required for each project. Each line in this file represents a resource required by a project:

Feature | Description 
----------|---------------
**`id`** | A `project_id` value from the `train.csv` file.  **Example:** `p036502`   
**`description`** | Desciption of the resource. **Example:** `Tenor Saxophone Reeds, Box of 25`   
**`quantity`** | Quantity of the resource required. **Example:** `3`   
**`price`** | Price of the resource required. **Example:** `9.95`   

**Note:** Many projects require multiple resources. The `id` value corresponds to a `project_id` in train.csv, so you use it as a key to retrieve all resources needed for a project:

The data set contains the following label (the value you will attempt to predict):

Label | Description
----------|---------------
`project_is_approved` | A binary flag indicating whether DonorsChoose approved the project. A value of `0` indicates the project was not approved, and a value of `1` indicates the project was approved.

### Notes on the Essay Data

<ul>
Prior to May 17, 2016, the prompts for the essays were as follows:
<li>project_essay_1: "Introduce us to your classroom"</li>
<li>project_essay_2: "Tell us more about your students"</li>
<li>project_essay_3: "Describe how your students will use the materials you're requesting"</li>
<li>project_essay_3: "Close by sharing why your project will make a difference"</li>
</ul>


<ul>
Starting on May 17, 2016, the number of essays was reduced from 4 to 2, and the prompts for the first 2 essays were changed to the following:<br>
<li>project_essay_1: "Describe your students: What makes your students special? Specific details about their background, your neighborhood, and your school are all helpful."</li>
<li>project_essay_2: "About your project: How will these materials make a difference in your students' learning and improve their school lives?"</li>
<br>For all projects with project_submitted_datetime of 2016-05-17 and later, the values of project_essay_3 and project_essay_4 will be NaN.
</ul>


In [1]:
import warnings
warnings.filterwarnings('ignore')

### Reading the data

In [2]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import re

In [3]:
project_df = pd.read_csv(filepath_or_buffer='train_data.csv')

In [4]:
print("The total number of data points in train data: {}".format(project_df.shape))

The total number of data points in train data: (109248, 17)


In [5]:
resource_df = pd.read_csv(filepath_or_buffer='resources.csv')

In [6]:
print("The total number of data points in resources data: {}".format(resource_df.shape))

The total number of data points in resources data: (1541272, 4)


### Preprocessing helping functions

In [7]:
def helper1(string):
    """
    This function helps in preprocessing.
    """
    string = string.lower()
    string = string.replace(' ', '_').replace('-', '_').replace('.', '')
    return string

In [8]:
def helper2(string):
    """
    This function helps in preprocessing.
    """
    string = string.lower()
    string = string.replace(' the ', '').replace(' ', '').replace('&', '_').replace(',', '_')
    return string

Reference: https://stackoverflow.com/a/47091490/4084039

In [9]:
def decontracted(string):
    """
    This function is used to handle contractions.
    """
    # specific
    string = re.sub(r"won\'t", "will not", string)
    string = re.sub(r"can\'t", "can not", string)
    # general
    string = re.sub(r"n\'t", " not", string)
    string = re.sub(r"\'re", " are", string)
    string = re.sub(r"\'s", " is", string)
    string = re.sub(r"\'d", " would", string)
    string = re.sub(r"\'ll", " will", string)
    string = re.sub(r"\'t", " not", string)
    string = re.sub(r"\'ve", " have", string)
    string = re.sub(r"\'m", " am", string)
    return string

Reference: https://gist.github.com/sebleier/554280

In [10]:
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you',
            "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself',
            'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her',
            'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them',
            'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom',
            'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are',
            'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having',
            'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if',
            'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for',
            'with', 'about', 'against', 'between', 'into', 'through', 'during',
            'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down',
            'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then',
            'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any',
            'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only',
            'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will',
            'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm',
            'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't",
            'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
            "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn',
            "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
            'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won',
            "won't", 'wouldn', "wouldn't"]

In [11]:
def preprocess_text(string):
    string = decontracted(string=string)
    string = string.replace('\\r', ' ').replace('\\n', ' ').replace('\\"', ' ')
    string = re.sub('[^A-Za-z0-9]+', ' ', string)
    string = ' '.join(e for e in string.split() if e.lower() not in stopwords)
    string = string.lower().strip()
    return string

### Preprocessing the columns

#### `project_grade_category`

- Remove the spaces ' '
- Replace the - with the _
- Convert all the letters to small

In [12]:
project_df['project_grade_category'] = project_df['project_grade_category'].apply(func=helper1)

In [13]:
display(project_df['project_grade_category'].value_counts().to_frame())

,project_grade_category
grades_prek_2,44225
grades_3_5,37137
grades_6_8,16923
grades_9_12,10963


#### `project_subject_categories`

- Remove the spaces ' ' and 'the'
- Replace the &, the , with the _
- Convert all the letters to small

In [14]:
project_df['project_subject_categories'] = project_df['project_subject_categories'].apply(func=helper2)

In [15]:
display(project_df['project_subject_categories'].value_counts().to_frame().head())

,project_subject_categories
literacy_language,23655
math_science,17072
literacy_language_math_science,14636
health_sports,10177
music_arts,5180


#### `teacher_prefix`

- Remove the dots .
- Convert all the letters to small

In [16]:
display(project_df['teacher_prefix'].value_counts().to_frame())

,teacher_prefix
Mrs.,57269
Ms.,38955
Mr.,10648
Teacher,2360
Dr.,13


In [17]:
project_df['teacher_prefix'] = project_df['teacher_prefix'].fillna(value='Mrs.')

In [18]:
display(project_df['teacher_prefix'].value_counts().to_frame())

,teacher_prefix
Mrs.,57272
Ms.,38955
Mr.,10648
Teacher,2360
Dr.,13


In [19]:
project_df['teacher_prefix'] = project_df['teacher_prefix'].apply(func=helper1)

In [20]:
display(project_df['teacher_prefix'].value_counts().to_frame())

,teacher_prefix
mrs,57272
ms,38955
mr,10648
teacher,2360
dr,13


#### `project_subject_subcategories`

- Remove the spaces ' ' and 'the'
- Replace the &, the , with the _
- Convert all the letters to small

In [21]:
project_df['project_subject_subcategories'] = project_df['project_subject_subcategories'].apply(func=helper2)

In [22]:
display(project_df['project_subject_subcategories'].value_counts().to_frame().head())

,project_subject_subcategories
literacy,9486
literacy_mathematics,8325
literature_writing_mathematics,5923
literacy_literature_writing,5571
mathematics,5379


#### `school_state`

In [23]:
project_df['school_state'] = project_df['school_state'].str.lower()

In [24]:
display(project_df['school_state'].value_counts().to_frame().head())

,school_state
ca,15388
tx,7396
ny,7318
fl,6185
nc,5091


#### `project_title`

- Remove the stopwords
- Clean the text

In [25]:
project_df['project_title'] = project_df['project_title'].apply(func=preprocess_text)

In [26]:
display(project_df['project_title'].to_frame().head())

,project_title
0,educational support english learners home
1,wanted projector hungry learners
2,soccer equipment awesome middle school students
3,techie kindergarteners
4,interactive math tools


#### `essay`

- Remove the stopwords
- Clean the text

In [27]:
project_df['essay'] = (project_df['project_essay_1'].map(str) + 
                       project_df['project_essay_2'].map(str) + 
                       project_df['project_essay_3'].map(str) + 
                       project_df['project_essay_4'].map(str))

In [28]:
project_df['essay'] = project_df['essay'].apply(func=preprocess_text)

In [29]:
display(project_df['essay'].to_frame().head())

,essay
0,students english learners working english seco...
1,students arrive school eager learn polite gene...
2,true champions not always ones win guts mia ha...
3,work unique school filled esl english second l...
4,second grade classroom next year made around 2...


#### `price`

In [30]:
price_df = resource_df.groupby(by='id').agg(dict(price='sum', quantity='sum')).reset_index()
display(price_df.head())

,id,price,quantity
0,p000001,459.56,7
1,p000002,515.89,21
2,p000003,298.97,4
3,p000004,1113.69,98
4,p000005,485.99,8


In [31]:
project_df = pd.merge(left=project_df, right=price_df, on='id', how='left')

In [32]:
display(project_df['price'].to_frame().head())

,price
0,154.60
1,299.00
2,516.85
3,232.90
4,67.98


### Column standardization

In [33]:
from sklearn.preprocessing import StandardScaler

In [34]:
scaler = StandardScaler()

In [35]:
project_df['std_price'] = scaler.fit_transform(X=project_df['price'].values.reshape(-1, 1))

In [36]:
display(project_df['std_price'].to_frame().head())

,std_price
0,-0.390533
1,0.002396
2,0.595191
3,-0.177469
4,-0.626236


### Column MinMaxScaler

In [37]:
from sklearn.preprocessing import MinMaxScaler

In [38]:
scaler = MinMaxScaler()

In [39]:
project_df['nrm_price'] = scaler.fit_transform(X=project_df['price'].values.reshape(-1, 1))

In [40]:
display(project_df['nrm_price'].to_frame().head())

,nrm_price
0,0.015397
1,0.029839
2,0.051628
3,0.023228
4,0.006733


End of the file.